In [1]:
import pandas as pd
import sys
sys.path.append('..')
from lib import Node

In [2]:
df = pd.read_pickle('../data/parsed/pickled_data_test.pickle')

In [3]:
rows = [1, 30, 32, 43, 46]

In [4]:
members = df.loc[rows]['Label'].values

In [5]:
members

array(['-1', '-1', '-1', '-1', '-1'], dtype=object)

In [6]:
members == '-1'

array([ True,  True,  True,  True,  True], dtype=bool)

In [7]:
a = members[members == '-1']
a

array(['-1', '-1', '-1', '-1', '-1'], dtype=object)

TODO:

- Dates to some kind of number
- from, to: split using membership
- dates: split on number
- subject: find subjects that are equal, or lsa matrix


In [8]:
one = df.loc[df['Scenario'] == '401']
one_labeled = one.loc[one['Label'] != '-1']
one_labeled.shape

(710, 21)

In [9]:
df['From'].apply(get_first).unique()

NameError: name 'get_first' is not defined

In [10]:
#find all unique from addresses
def get_first(l):
    return l[0]
def long_func():
    uniques = one['From'].apply(get_first).unique()[:5]
    rows = [x for x in range(one.shape[0])]
    features = [x for x in range(one.shape[1] - 2)]
    depth = 0
    max_depth = 4
    #for each from address:
    best_gini = 2
    best_address = ''

    count = 0
    for address in uniques:
        print(address)

        #'split' on that address
        this_sender_rows = one.loc[one['From'].apply(get_first) == address].index.values
        other_sender_rows = one.loc[one['From'].apply(get_first) != address].index.values

        #maybe a little sketch to not remove this feature?
        from_this_address = Node(one, this_sender_rows, features, depth + 1, max_depth)
        from_other_address = Node(one, other_sender_rows, features, depth + 1, max_depth)

        #record gini value
        curr_gini = Node.aggregate_gini(from_this_address.calc_gini_index(), from_other_address.calc_gini_index(),
                                       len(from_this_address.rows), len(from_other_address.rows))

        #take the best gini value
        if curr_gini < best_gini:
            best_gini = curr_gini
            best_address = address
        count += 1
        print(float(count) / len(uniques))

In [11]:
%prun long_func()

david.allan@enron.com
0.2
stacey.wales@enron.com
0.4
pppcmail@pppc.org
0.6
pete.davis@enron.com
0.8
john.brindle@enron.com
1.0
 

In [ ]:
best_address